## **Import the libraries** ##

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
df_web_test=pd.read_csv("../Data/Clean/df_final_web_data_test_clean.txt")

## **Create the files** ##
Dont need to run this part again

In [ ]:
#Create the dataframes
df_clients = pd.read_csv("../Data/Raw/df_final_experiment_clients.txt")
df_demo = pd.read_csv("../Data/Raw/df_final_demo.txt")
df_web1 = pd.read_csv("../Data/Raw/df_final_web_data_pt_1.txt")
df_web2 = pd.read_csv("../Data/Raw/df_final_web_data_pt_2.txt")
df_clients.head()

In [ ]:
#Concat data_web
df_web=pd.concat([df_web1,df_web2], axis=0)
df_web=df_web.reset_index(drop=True)
df_web

In [ ]:
#Merge demo with clients
df_demo=pd.merge(df_demo,df_clients, on="client_id", how="left")



In [ ]:
#I create demo_control
conditioncontrol=df_demo["Variation"]=="Control"
df_demo_control=df_demo[conditioncontrol]
df_demo_control

In [ ]:
#I create demo_test
conditiontest=df_demo["Variation"]=="Test"
df_demo_test=df_demo[conditiontest]
df_demo_test

In [ ]:
#I add information about the clientes to data_web
df_web=pd.merge(df_web,df_clients, on="client_id", how="left")
df_web

In [ ]:
#Create web_test
conditiontest=df_web["Variation"]=="Test"
df_web_test=df_web[conditiontest]
df_web_test

In [ ]:
#Create web_control
conditioncontrol=df_web["Variation"]=="Control"
df_web_control=df_web[conditioncontrol]
df_web_control

In [ ]:
#FINAL RESULT 4 FILES:
#df_web_control
#df_web_test
#df_demo_control
#df_demo_test
df_web_test.to_csv("../Data/Raw/df_final_web_data_test.txt")

## **Cleaning and columns aggregation** ##

In [ ]:
#Clean web_data_test 1148 duplicated values
df_web_test = pd.read_csv("../Data/Raw/df_final_web_data_test.txt")
df_web_test=df_web_test.reset_index(drop=True)
df_web_test.duplicated().sum()
df_web_test.drop_duplicates(inplace=True)
df_web_test.duplicated().sum()

In [ ]:
#Split date and time
df_web_test["date"]=df_web_test["date_time"].str.split(" ", expand=True)[0]
df_web_test["time"]=df_web_test["date_time"].str.split(" ", expand=True)[1]
df_web_test["hour"]=df_web_test["time"].str.split(":",expand=True)[0]
df_web_test["hour"]=df_web_test["hour"].astype(int)
df_web_test.head()

In [ ]:
#Change the dtypes of the column time %H,%M,%S format
df_web_test['time'] = (pd.to_datetime(df_web_test['time'], format='%H:%M:%S'))
df_web_test

In [ ]:
#Change the values of process_step
df_web_test["process_step"]=df_web_test["process_step"].map({"start":0,"step_1":1,"step_2":2,"step_3":3,"confirm":4})
df_web_test["process_step"].unique()

In [ ]:
#Create two new columns in order to have the step_jump and one column "control" to filter when the data is not from the same visit_id.
df_web_test=df_web_test.sort_values(["visit_id","date", "time", "process_step"])
df_web_test["step_jump"]=df_web_test["process_step"].astype(str).shift(+1)+"-"+df_web_test["process_step"].astype(str)
#Create column control which help me to determine, later, is the steps follows a orded sense.
df_web_test["control"]=(df_web_test["process_step"]-df_web_test["process_step"].shift(+1)).abs()
df_web_test["dif_time"]=(df_web_test["time"])-(df_web_test["time"].shift(+1))
df_web_test

In [ ]:
#Create a column for the total of segs spend between steps
df_web_test=df_web_test.dropna(subset=["control"])
df_web_test["dif_time"] = (df_web_test['dif_time'].astype(str))
df_web_test["dif_time"]=(df_web_test["dif_time"].str.split(" ")).str[-1]
df_web_test["dif_hour"]=((df_web_test["dif_time"].str.split(":")).str[0])
df_web_test["dif_hour"]=df_web_test["dif_hour"].astype(int)
df_web_test["dif_min"]=((df_web_test["dif_time"].str.split(":")).str[1]).astype(int)
df_web_test["dif_seg"]=((df_web_test["dif_time"].str.split(":")).str[2]).astype(int)
df_web_test["total_seg"]=df_web_test["dif_hour"]*3600 + df_web_test["dif_min"]*60 + df_web_test["dif_seg"]
df_web_test

In [ ]:
#Function for the keep_control column. It is only going to be used in terms of TIME BETWEEN STEPS
def row_count(df):
    df['next_visit'] = df['visit_id'].shift(+1) 
    df['keep_contol'] = df.apply(lambda row: "YES" if row['visit_id'] == row['next_visit'] else "NO", axis=1)
    df.drop(columns=['next_visit'], inplace=True)
    return df

In [ ]:
#Add the column keep_control to my DataFrame
df_web_test=row_count(df_web_test)
df_web_test
df_web_test.to_csv("../Data/Clean/df_final_web_data_test_clean.txt")
#FINAL DATA FRAME TO USE: df_web_test

## **Analysis** ##

## **Device** ##

In [ ]:
#Create a new date_frame to see the use of device
df_web_test_device=df_web_test[["client_id","visitor_id"]]
df_web_test_device=df_web_test_device.drop_duplicates()
df_web_test_device=pd.DataFrame(df_web_test_device.groupby("client_id")["visitor_id"].count())
df_web_test_device=df_web_test_device.rename(columns={"visitor_id":"num_devices"})
df_web_test_device=df_web_test_device.reset_index()
df_web_test_device

In [ ]:
device_stat=df_web_test_device["num_devices"].describe()
device_mode=df_web_test_device["num_devices"].mode()[0]

print(f"The stadictical for the number of device per client is {device_stat}")
print(f"The mode for the number of device pero cliente is {device_mode}")

In [ ]:
sns.histplot(df_web_test_device['num_devices'], kde=True, bins=15, color="orange")
plt.show()

**Analysis**: from all the almost 27.000 clients, almost 75% of the customer only use 1 device to get into our portal.There is some clients who use more than 4 devices that we can consider outliners.

In [ ]:
sns.boxplot(df_web_test_device['num_devices'], color="lightblue")
plt.show()

## **Connection times** ##

In [ ]:
#Create a new date_frame to see how many interaction has been made for each client
df_web_test_visit=df_web_test[["client_id","visit_id"]]
df_web_test_visit=df_web_test_visit.drop_duplicates()
df_web_test_visit=pd.DataFrame(df_web_test_visit.groupby("client_id")["visit_id"].count())
df_web_test_visit=df_web_test_visit.rename(columns={"visit_id":"num_visits"})
df_web_test_visit=df_web_test_visit.reset_index()
df_web_test_visit

In [ ]:
visit_stat=df_web_test_visit["num_visits"].describe()
visit_mode=df_web_test_visit["num_visits"].mode()[0]

print(f"The stadictical for the number of device per client is {visit_stat}")
print(f"The mode for the number of device pero cliente is {visit_mode}")

In [ ]:
sns.histplot(df_web_test_visit['num_visits'], kde=True, bins=15, color="red")
plt.show()

In [ ]:
sns.boxplot(df_web_test_visit['num_visits'], color="red")
plt.show()

**Analysis**: The test clients have only interact one time with the new web. As we can see the mean of interactions is 1.37 and only 25% of the total clients have interact with the web, at least, 2 times. We also see some outliner which has interact with the web more than 4 times.

## **STEPS**

In [ ]:
df_web_test["process_step"].describe()

In [ ]:
sns.histplot(df_web_test['process_step'], kde=True, bins=60, color="green")
plt.show()

**Analysis**: The web is a little bit confusing. The clients, as average, do not reach the second step and they get stack in the step 1. Less than 25% of the clients reach the final step. But all the clients which reach the setp 3 get to finish the whole process. So the more confusing part are in order to get to step 1 as we can see more difference in the histoplot. The difficulty decrese as the steps go ahead.

## **Hour**

In [ ]:
df_web_test_hour=df_web_test.sort_values("hour", ascending=True)
df_web_test_hour=df_web_test_hour[["visit_id","hour"]]
df_web_test_hour=df_web_test_hour.drop_duplicates()
df_web_test_hour

In [ ]:
df_web_test_hour["hour"].describe()

In [ ]:
sns.histplot(df_web_test_hour['hour'], kde=True, bins=20, color="green")
plt.show()

In [ ]:
sns.boxplot(df_web_test_hour['hour'], color="lightgreen")
plt.show()

**Analysis**: More of the interactis has been made between 10 am to 15 pm. There is just some outlines which access to the 12 am 

## **Time between steps** ##

In [ ]:
#Filter what we want: control condition 1, steps forward 0-1;1-2;2-3;3-4. keep_control=YES
condition_control=df_web_test["control"]==1
df_web_test_dif_time=df_web_test[condition_control]
condition_step=(df_web_test_dif_time["step_jump"]=="0-1") | (df_web_test_dif_time["step_jump"]=="1-2") | (df_web_test_dif_time["step_jump"]=="2-3") | (df_web_test_dif_time["step_jump"]=="3-4")
df_web_test_dif_time=df_web_test_dif_time[condition_step]
condition_keep=df_web_test_dif_time["keep_contol"]=="YES"
df_web_test_dif_time=df_web_test_dif_time[condition_keep]
df_web_test_dif_time
df_web_test_dif_time

In [ ]:
df_web_test_dif_time["total_seg"].describe()

In [ ]:
time_stat=df_web_test_dif_time.groupby("step_jump")["total_seg"].agg(["count","median","std"])
time_stat

In [ ]:
test=df_web_test.groupby("process_step")["process_step"].value_counts()
test

In [ ]:
df_web_test

In [ ]:
sns.histplot(df_web_test_dif_time['total_seg'], kde=True, bins=60, color="green")
plt.show()

In [ ]:
sns.boxplot(df_web_test_dif_time['total_seg'], color="lightgreen")
plt.show()

**Analysis**: As there is lot of outliners which we considers clients who did not not close the session and remain the PC open, we are going to use the median instead of mean. 
The total process, as median, it takes 152 minutes, being the first step the faster one with 10 seconds and the longest is number 2 with 66 seconds.

## **Completion ratio** ##

In [ ]:
#The completion ratio is the number of clients who made it to step 4 out of the total number of visist
#Calculate the number of clients who made it to step 4
step_4=(df_web_test["process_step"]==4).value_counts()
step_4


In [ ]:
completion_ratio=(step_4[True])/(df_web_test.visit_id.nunique())

print(f"The completion ratio is {completion_ratio}. That means, out of {df_web_test.visit_id.nunique()}, a total of {completion_ratio * 100}% has made it to the final step")

## **Error rates**

We consider an error when the client go backwards in the logical step process, that means if the client from process 1 to 0, from 2 to 1 or from 3 to 2. 
We will caculate first the total amount of these errors and for the rate we will take into account the total number movements. That means, if 1 client made it to step 4 it will count as 4 movemnets: from 0 to 1, from 1 to 2, for 2 to 3 and from 3 to 4.

In [ ]:
#First we filter the values that we are going to use. Filter: Control=1; Keep=YES.
#We only want the errors in the process. 
condition_control=df_web_test["control"]==1
df_error_rate=df_web_test[condition_control]
condition_keep=df_error_rate["keep_contol"]=="YES"
df_error_rate=df_error_rate[condition_keep]
condition_step=(df_error_rate["step_jump"]=="1-0") | (df_error_rate["step_jump"]=="2-1") | (df_error_rate["step_jump"]=="3-2") | (df_error_rate["step_jump"]=="4-3")
df_error_rate=df_error_rate[condition_step]
df_error_rate

In [ ]:
sns.histplot(df_error_rate['step_jump'], kde=True, bins=15, color="blue")
plt.show()

In [ ]:
#We display the number of errors per steps
error_counts=pd.DataFrame(df_error_rate["step_jump"].value_counts())
error_counts

In [ ]:
sns.histplot(df_error_rate.sort_values("step_jump")['step_jump'], kde=True, bins=20, color="green")
plt.show()

In [ ]:
sns.boxplot(df_error_rate.sort_values("step_jump")['step_jump'], color="lightgreen")
plt.show()

**Analysis**: we can see that there is only 1 outliner, which is going back from step 4, so the clients do understand that the process is finished once they get to number 4. The steps that have more rate of error is from step 1 to 0 and that number decrease as the client continues in the process. That make sense as the number of clients which complete each step also decrease in the same trend. 

We compare the both histoplot bellow. 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5)) 
sns.histplot(df_web_test['process_step'], kde=True,ax=axes[0], bins=15, color="blue")
sns.histplot(df_error_rate.sort_values("step_jump")['step_jump'], kde=True,ax=axes[1], bins=20, color="green")
plt.tight_layout()
plt.show()

In [ ]:
#Number of error
error_counts["count"].sum()

In [ ]:
#Total number of steps jump
(df_web_test["visit_id"].value_counts()).sum()

In [ ]:
ratio_error=((df_error_rate["step_jump"].value_counts()).sum())/((df_web_test["visit_id"].value_counts()).sum())

print(f"The ratio erros is {ratio_error}, that means that out of the {(df_web_test["visit_id"].value_counts()).sum()} step jumps, {ratio_error *100}% have been error")